In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
import numpy as np
from Scratch import metadata_loader

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
md = md[md["distance"] == max_distance]
md = md.sort_values(by='backend_name', ascending=False)

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
76,2023-10-29 14:47:58.814875+01:00,bigger_rep_codes,ibm_sherbrooke,cmz653m3r3vg008wf9j0,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30.0,1,_is_hex=True,Run bigger Repetition codes
75,2023-10-29 14:47:43.903639+01:00,bigger_rep_codes,ibm_sherbrooke,cmz64zvvcq70008qdxcg,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30.0,0,_is_hex=True,Run bigger Repetition codes


In [4]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memories

{'mmr_log_1': array([[ -9827335.+1.37223950e+07j, -53977778.+7.99818970e+07j,
         -79071209.-9.83478680e+07j, ...,  -8432334.-7.54241200e+06j,
          -5175264.-8.50117500e+06j,   6540639.-5.34872300e+06j],
        [-10489210.+2.02035840e+07j, -83693508.+7.02712640e+07j,
         -68739961.-8.00257260e+07j, ...,  -6971682.-7.36254800e+06j,
          -4029180.-5.24985200e+06j,  12069718.-1.06493800e+07j],
        [-14341855.+1.95327160e+07j, -72534674.+6.55941800e+07j,
         -67845350.-1.14758427e+08j, ...,  -6694234.-5.57350800e+06j,
         -10807815.-1.09097600e+07j,   7684249.-1.06278620e+07j],
        ...,
        [ -8225117.+9.95315900e+06j, -71635045.+6.52791880e+07j,
         -62033555.-8.20048330e+07j, ...,   6253861.-8.49423500e+06j,
           5556042.-8.86801300e+06j,   8736877.-1.35674410e+07j],
        [-16130944.+1.36266200e+07j, -57588956.+5.27301930e+07j,
         -58264550.-6.76114050e+07j, ...,  -7844011.-9.03883700e+06j,
          -8234982.-9.29126300e+06j

In [5]:
from soft_info import get_repcode_layout, get_KDEs, get_counts
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)
print(kde_dict)

code = RepetitionCodeCircuit(max_distance, max_distance)
decoder = UnionFindDecoder(code)

from qiskit_qec.analysis._c_analysis import _CErrorPropagator             failed, raising No module named 'qiskit_qec.analysis._c_analysis'
Traceback (most recent call last):
  File "/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-qec/src/qiskit_qec/analysis/extensions.py", line 21, in <module>
    from qiskit_qec.analysis._c_analysis import _CErrorPropagator  # pylint: disable=unused-import
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'qiskit_qec.analysis._c_analysis'
from qiskit_qec.analysis._c_analysis import _CFaultEnumerator             failed, raising No module named 'qiskit_qec.analysis._c_analysis'
Traceback (most recent call last):
  File "/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-qec/src/qiskit_qec/analysis/extensions.py", line 32, in <module>
    from qiskit_qec.analysis._c_analysis import _CFaultEnumerator  # pylint: disable

{114: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 116: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 118: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 120: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 122: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 104: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 102: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 100: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 98: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 96: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 94: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 75: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 77: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 79: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 81: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 8

In [6]:
from soft_info import soft_reweight, get_counts
from tqdm import tqdm

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))):    
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    decoder.graph = reweighted_graph

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    print("\nmeas_str:", count_key)
    print("flipped_qubit_dict", flipped_qubit_dict)
    print("logical:", logical, "\n")
    if logical[0] == 1:
        print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


  0%|          | 1/1111 [00:02<37:28,  2.03s/it]


meas_str: 111101000001001001110000100000 11100010011111011001000000000 11111110010101111001000110000 11111010011111011000000100000 11111010010101110001000010000 11111100011111011000001100000 11111000011001110001000110000 11111100011111010000000000000 10111110001001011001000101000 00111100001111111000000100000 11100100000101011001001101000 00111000001111111000000000000 11100110010101011001000101000 00100010001111111000000000000 11100010001001111001000101000 00100000001111010000000000000 01100010001001010001000101000 01100110001111001100010000000 11100110000101101001000101000 01100000001111000100000000000 11100110001011100001000101000 11100110001011000100000000000 01100110001011000010000011000 00100100001011000100000000000 01100010001011000010000111000 10100100001011000100000010000 11100000001001000010000111000 10100010001011000100000100000 01000000001001000110000100000 11000010000000000000000000000 00000000000000000000000000000
flipped_qubit_dict {'Cluster_0': [], 'Cluster_1': [11], 'C

  0%|          | 2/1111 [00:04<37:39,  2.04s/it]


meas_str: 000010100011111110000000100000 00111000000011001000001000010 00110110010011001000001110010 10110000000011001000001110000 10110110010011001000001000000 01110110000011101100001011000 11010000010011000000001000000 01101110000011101101001011000 10011110010011001001001000000 00101010000011101101001000000 10000110010011001001001000000 00101000000011101101000100000 01001100000011001001001000000 00110010001011101101000100000 11101100000011011001001000000 01011000001011101101000100000 10001100000011011001001000000 01000110001011001101000100000 11001110000001111101001000000 10000100001011000101000100000 01001110000001011101001000000 00000100001011011101000100000 11001000000011110000000100000 00000110001011110100000100000 00000000000001101000000100000 00000110001011101100000100000 00000000000001000000000111000 00000110000001000000000100000 00000000000001000000000100000 00000100000001000100000100000 00000000000000000000000100000
flipped_qubit_dict {'Cluster_0': [], 'Cluster_1': [], 'Clu

  0%|          | 3/1111 [00:06<37:48,  2.05s/it]

...


meas_str: 100010100001110100010111101000 01001011000011000001000011001 11010101001011110010100000101 01001101000011100001000011011 11010111001011110110100011111 01001000000011100001000011000 11010110001011110010100001101 01001110010011100001000001000 11011000001011010010100011101 00100110000011000001000001000 11011010001011110010100010101 00100100000111110001000011000 11011100001111000010101100111 11110010000111110001000011011 11001100001001010010101101111 10110100000011100001001100010 11101101001101010010101101110 10110101000001100001001100010 01101011001101010010101101010 00110111000001100000000001010 01101001001101010000101101001 00110111000001100000000100010 01101001001101010000000001000 01010101001101100000000100000 01001101001101010000000001000 00011111001001100000000100000 01001101001001010000000001000 00010001001001000000000110000 11001011001001100000000000000 00010010001001000000000000000 11001010001000000000000000000
flipped_qubit_dict {'Cluster_0': [], 'Cluster_1': [], 'Clu

 77%|███████▋  | 855/1111 [40:27<10:59,  2.58s/it]


meas_str: 100010101010000101111000000100 10001101010110001111000111010 11010000101110110111100111100 00001100010110001111000001010 11010000101010110011100110101 01101101001010001001000101010 11110110101010110011100101101 01011101001000001101000101010 10101000101000110011100101101 11011101001001100001000101010 11101000111000110011100101101 11111101011001100001000100010 11001000111001010000100110010 11111001011001100001000001010 10001010101001010000100111101 10111001001001100001000001110 00001010101001010000100111101 11011001001001100001000110110 00001010101001100000100110101 11011011001001100001000110010 00001000101001100000100111011 11011111001001100000000110010 11000010101001100000000001011 01011101001001100000000110011 00000100101001000000000001001 01011001001001000000000000011 10000010101001000000000001010 01000011001001000000000000000 11000010101001100000000001110 00100010001001000000000000100 01000000001001000000000001110
flipped_qubit_dict {'Cluster_0': [2], 'Cluster_1': [3], 'C

 77%|███████▋  | 855/1111 [40:27<12:06,  2.84s/it]


KeyboardInterrupt: 

In [ ]:
from soft_info import soft_reweight, get_counts
from tqdm import tqdm

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))):    
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    #reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    #decoder.graph = reweighted_graph

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    print("\nmeas_str:", count_key)
    print("flipped_qubit_dict", flipped_qubit_dict)
    print("logical:", logical, "\n")
    if logical[0] == 1:
        print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)
